### Connect to the DuckDB warehouse (Day-1 DB file)

In [1]:
from pathlib import Path
import duckdb

project_root = Path.cwd().resolve()
while not (project_root / "Day-1").exists():
    if project_root == project_root.parent:
        raise FileNotFoundError("Could not find project root containing Day-1.")
    project_root = project_root.parent

db_path = project_root / "Day-1" / "data" / "warehouse" / "day1.duckdb"
print("Project root:", project_root)
print("DB path:", db_path, "exists?", db_path.exists())

con = duckdb.connect(str(db_path))
print("Connected.")
print("Tables:", [t[0] for t in con.execute("SHOW TABLES").fetchall()])


Project root: C:\Users\sarfo\Dropbox\Courses\Data Science\30-days-of-data-science
DB path: C:\Users\sarfo\Dropbox\Courses\Data Science\30-days-of-data-science\Day-1\data\warehouse\day1.duckdb exists? True
Connected.
Tables: ['bronze_diabetes', 'gold_diabetes_base', 'gold_diabetes_features_v1', 'silver_diabetes', 'silver_diabetes_typed']


### Load the modeling table into pandas

In [2]:
import pandas as pd

df = con.execute("SELECT * FROM gold_diabetes_features_v1").df()
print(df.shape)
df.head()


(101766, 20)


,encounter_id,person_id,label,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,diabetesMed,change,insulin,A1Cresult
0,2278392,8222157,0,1,41,0,1,0,0,0,Caucasian,Female,[0-10),6,25,1,No,No,No,None
1,149190,55629189,0,3,59,0,18,0,0,0,Caucasian,Female,[10-20),1,1,7,Yes,Ch,Up,None
2,64410,86047875,0,2,11,5,13,2,0,1,AfricanAmerican,Female,[20-30),1,1,7,Yes,No,No,None
3,500364,82442376,0,2,44,1,16,0,0,0,Caucasian,Male,[30-40),1,1,7,Yes,Ch,Up,None
4,16680,42519267,0,1,51,0,8,0,0,0,Caucasian,Male,[40-50),1,1,7,Yes,Ch,Steady,None


### Missingness overview (rates per column)

This is the first thing you do on any messy healthcare dataset.

In [3]:
import numpy as np

miss_rate = df.isna().mean().sort_values(ascending=False)
miss_count = df.isna().sum().sort_values(ascending=False)

missing_summary = pd.DataFrame({
    "missing_count": miss_count,
    "missing_rate": miss_rate
})

missing_summary


,missing_count,missing_rate
race,2273,0.022336
person_id,0,0.000000
insulin,0,0.000000
change,0,0.000000
diabetesMed,0,0.000000
admission_source_id,0,0.000000
discharge_disposition_id,0,0.000000
admission_type_id,0,0.000000
age,0,0.000000
gender,0,0.000000


Save it (so Day 3 has an artifact):

In [4]:
reports_dir = project_root / "Day-3" / "reports"
reports_dir.mkdir(parents=True, exist_ok=True)

missing_summary.to_csv(reports_dir / "DAY03_missingness_summary.csv", index=True)
print("Saved:", reports_dir / "DAY03_missingness_summary.csv")


Saved: C:\Users\sarfo\Dropbox\Courses\Data Science\30-days-of-data-science\Day-3\reports\DAY03_missingness_summary.csv


### Missingness vs outcome (quick MCAR sanity check)

If missingness were MCAR, the label prevalence should look similar in “missing” vs “observed” groups. Big differences are a red flag (often MAR/MNAR-ish).

In [5]:
y = df["label"].astype(int)

rows = []
for col in df.columns:
    if col in ["encounter_id", "person_id", "label"]:
        continue
    m = df[col].isna()
    if m.sum() == 0:
        continue
    prev_missing = float(y[m].mean())
    prev_observed = float(y[~m].mean())
    rows.append({
        "column": col,
        "missing_count": int(m.sum()),
        "missing_rate": float(m.mean()),
        "prev_if_missing": prev_missing,
        "prev_if_observed": prev_observed,
        "diff_missing_minus_observed": prev_missing - prev_observed
    })

miss_vs_y = pd.DataFrame(rows).sort_values("missing_rate", ascending=False)
miss_vs_y


,column,missing_count,missing_rate,prev_if_missing,prev_if_observed,diff_missing_minus_observed
0,race,2273,0.022336,0.08271,0.112259,-0.029549


Save it:

In [6]:
miss_vs_y.to_csv(reports_dir / "DAY03_missingness_vs_label.csv", index=False)
print("Saved:", reports_dir / "DAY03_missingness_vs_label.csv")


Saved: C:\Users\sarfo\Dropbox\Courses\Data Science\30-days-of-data-science\Day-3\reports\DAY03_missingness_vs_label.csv


### Leakage-safe split (same logic as Day 2)

In [7]:
from sklearn.model_selection import GroupShuffleSplit

groups = df["person_id"]
gss = GroupShuffleSplit(n_splits=1, test_size=0.20, random_state=42)
train_idx, test_idx = next(gss.split(df, y, groups=groups))

train_df = df.iloc[train_idx].copy()
test_df  = df.iloc[test_idx].copy()

overlap = set(train_df["person_id"]).intersection(set(test_df["person_id"]))
print("Train rows:", train_df.shape[0], "Test rows:", test_df.shape[0])
print("Train prevalence:", float(train_df["label"].mean()))
print("Test prevalence:", float(test_df["label"].mean()))
print("Patient overlap:", len(overlap))  # must be 0


Train rows: 81613 Test rows: 20153
Train prevalence: 0.11280065675811451
Test prevalence: 0.10673348881059892
Patient overlap: 0


### Two baselines: without vs with missingness indicators

Today’s key modeling idea is: missingness indicators can help when missingness is informative (often MAR/MNAR-ish), because “missingness itself” becomes a predictive signal.

#### Define columns

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

id_cols = ["encounter_id", "person_id", "label"]
feature_cols = [c for c in df.columns if c not in id_cols]

numeric_cols = [
    "time_in_hospital", "num_lab_procedures", "num_procedures", "num_medications",
    "number_outpatient", "number_emergency", "number_inpatient"
]
categorical_cols = [c for c in feature_cols if c not in numeric_cols]

X_train = train_df[feature_cols]
y_train = train_df["label"].astype(int)

X_test = test_df[feature_cols]
y_test = test_df["label"].astype(int)


#### Model A: standard imputation (no missing indicators)

In [9]:
num_pipe_A = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="median")),
    ("scale", StandardScaler())
])

cat_pipe_A = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

prep_A = ColumnTransformer(
    transformers=[
        ("num", num_pipe_A, numeric_cols),
        ("cat", cat_pipe_A, categorical_cols)
    ]
)

clf = LogisticRegression(max_iter=2000, solver="lbfgs")

pipe_A = Pipeline(steps=[("prep", prep_A), ("clf", clf)])
pipe_A.fit(X_train, y_train)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('prep', ...), ('clf', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains spa

#### Model B: same thing, but add missingness indicators

In [10]:
num_pipe_B = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="median", add_indicator=True)),
    ("scale", StandardScaler())
])

cat_pipe_B = Pipeline(steps=[
    ("impute", SimpleImputer(strategy="most_frequent", add_indicator=True)),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

prep_B = ColumnTransformer(
    transformers=[
        ("num", num_pipe_B, numeric_cols),
        ("cat", cat_pipe_B, categorical_cols)
    ]
)

pipe_B = Pipeline(steps=[("prep", prep_B), ("clf", clf)])
pipe_B.fit(X_train, y_train)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('prep', ...), ('clf', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains spa

#### Evaluate both models (PR-AUC, ROC-AUC, Brier, top-K)

In [11]:
import numpy as np
from sklearn.metrics import average_precision_score, roc_auc_score, brier_score_loss

def eval_probs(y_true, p):
    return {
        "prevalence": float(np.mean(y_true)),
        "mean_p": float(np.mean(p)),
        "median_p": float(np.median(p)),
        "pr_auc": float(average_precision_score(y_true, p)),
        "roc_auc": float(roc_auc_score(y_true, p)),
        "brier": float(brier_score_loss(y_true, p)),
    }

def topk_table(y_true, p, fracs=(0.01, 0.05, 0.10, 0.20)):
    y_true = np.asarray(y_true)
    rows = []
    n = len(y_true)
    order = np.argsort(-p)
    for frac in fracs:
        k = max(1, int(np.floor(frac*n)))
        idx = order[:k]
        rows.append({
            "top_frac": frac,
            "k": k,
            "captured": int(np.sum(y_true[idx])),
            "precision_at_k": float(np.mean(y_true[idx])),
            "threshold": float(np.quantile(p, 1-frac))
        })
    return pd.DataFrame(rows)

pA = pipe_A.predict_proba(X_test)[:, 1]
pB = pipe_B.predict_proba(X_test)[:, 1]

metrics_A = eval_probs(y_test, pA)
metrics_B = eval_probs(y_test, pB)

print("Model A (no indicators):", metrics_A)
print("Model B (with indicators):", metrics_B)

topA = topk_table(y_test, pA)
topB = topk_table(y_test, pB)

print("\nTop-K Model A:\n", topA)
print("\nTop-K Model B:\n", topB)


Model A (no indicators): {'prevalence': 0.10673348881059892, 'mean_p': 0.11115573850334283, 'median_p': 0.09391402218741597, 'pr_auc': 0.1993444353875619, 'roc_auc': 0.658420269022229, 'brier': 0.09188424740300427}
Model B (with indicators): {'prevalence': 0.10673348881059892, 'mean_p': 0.1111557385033444, 'median_p': 0.09391402218741343, 'pr_auc': 0.1993444353875619, 'roc_auc': 0.658420269022229, 'brier': 0.0918842474030044}

Top-K Model A:
    top_frac     k  captured  precision_at_k  threshold
0      0.01   201        73        0.363184   0.365170
1      0.05  1007       300        0.297915   0.232186
2      0.10  2015       514        0.255087   0.178544
3      0.20  4030       799        0.198263   0.135494

Top-K Model B:
    top_frac     k  captured  precision_at_k  threshold
0      0.01   201        73        0.363184   0.365170
1      0.05  1007       300        0.297915   0.232186
2      0.10  2015       514        0.255087   0.178544
3      0.20  4030       799        0.1982

Save Day 3 results

In [12]:
import json

out = {
    "split": {"method": "GroupShuffleSplit by person_id", "test_size": 0.20, "random_state": 42, "patient_overlap": 0},
    "model_A_no_indicators": metrics_A,
    "model_B_with_indicators": metrics_B,
    "topk_A": topA.to_dict(orient="records"),
    "topk_B": topB.to_dict(orient="records"),
}

with open(reports_dir / "DAY03_metrics.json", "w", encoding="utf-8") as f:
    json.dump(out, f, indent=2)

topB.to_csv(reports_dir / "DAY03_topk_with_indicators.csv", index=False)
print("Saved:", reports_dir / "DAY03_metrics.json")
print("Saved:", reports_dir / "DAY03_topk_with_indicators.csv")


Saved: C:\Users\sarfo\Dropbox\Courses\Data Science\30-days-of-data-science\Day-3\reports\DAY03_metrics.json
Saved: C:\Users\sarfo\Dropbox\Courses\Data Science\30-days-of-data-science\Day-3\reports\DAY03_topk_with_indicators.csv


### Close DuckDB

In [13]:
con.close()
print("DuckDB connection closed.")


DuckDB connection closed.
